In [1]:
import tensorflow as tf
from tensorflow.python.keras import regularizers
import pandas as pd
from tensorflow.python.ops import nn
import numpy as np
from utils import *
from models import *

if tf.__version__ != "2.0.0":
    tf.enable_eager_execution()
"""
    Hyper-parameters
"""


class HyperParameters:
    """
    Easy to manage HyperParameters
    """
    n_epochs = 4
    n_batches = 1000
    target_loss = 1e-5
    thresh_hold = 1e-5
    dataset = "mnist"
    regularizer = regularizers.l1(0.01)
    layer_type = "cnn"
    seed = 1234
    n_inputs = 28 * 28
    n_outputs = 10
    n_hiddens = 784
    n_filters = 64
    n_kernels = 3
    n_outputs = 100
    inp_shape = None

if __name__ == "__main__":
    params = HyperParameters
    """
        Load data
    """
    tf.random.set_seed(params.seed)
    x_train, y_train, x_test, y_test, params = get_data(params)

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(
        8192, seed=params.seed).batch(params.n_batches)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(
        8192, seed = params.seed).batch(params.n_batches)
    """
        Set model
    """
    if params.layer_type == "dense":
        model = DenseModel(params.n_hiddens, params.n_inputs, params.n_outputs)
    elif params.layer_type == "cnn":
        model = CNNModel(params.n_filters, params.n_kernels, params.n_outputs, params.inp_shape , params.regularizer)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    """
        Train data
    """
    epoch = 0
    while (epoch < params.n_epochs) or (epoch_loss_avg.result() < params.target_loss):
        epoch += 1
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_train_acc_avg = tf.keras.metrics.Mean()
        epoch_train_val_avg = tf.keras.metrics.Mean()
        for x, y in train_dataset:
            with tf.GradientTape() as tape:
                out = model(x)
                batch_loss = my_loss(out, y, params.n_outputs)
                variables = model.trainable_variables
                grads = tape.gradient(batch_loss, variables)
                optimizer.apply_gradients(zip(grads, variables))
            epoch_loss_avg(batch_loss)
        if epoch%1 == 0:
            print(model.summary())
        for x, y in test_dataset:
            test_out = model(x)
            accuracy = cal_acc(test_out, y)
            epoch_acc_avg(accuracy)
        model.sparsify_weights(params.thresh_hold)
        model.add_layer()
        print('Epoch : {} ----- Loss : {} ----- Acc : {}'.format(epoch, epoch_loss_avg.result(), epoch_acc_avg.result()))


Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Conv1D)           multiple                  5440      
_________________________________________________________________
cnn_input (Conv1D)           multiple                  579       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
classify (MyDenseLayer)      multiple                  850       
Total params: 6,869
Trainable params: 6,869
Non-trainable params: 0
_________________________________________________________________
None
Epoch : 1 ----- Loss : 0.5890613794326782 ----- Acc : 0.9041999578475952
Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Conv1D)      

In [2]:
model.summary()

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Conv1D)           multiple                  5440      
_________________________________________________________________
cnn_input (Conv1D)           multiple                  579       
_________________________________________________________________
cnn_1d_1 (Conv1D)            multiple                  12352     
_________________________________________________________________
cnn_1d_2 (Conv1D)            multiple                  12352     
_________________________________________________________________
cnn_1d_3 (Conv1D)            multiple                  12352     
_________________________________________________________________


ValueError: You tried to call `count_params` on cnn_1d_4, but the layer isn't built. You can build it manually via: `cnn_1d_4.build(batch_input_shape)`.

In [23]:
tf.math.in_top_k(tf.cast(tf.squeeze(y_test),tf.int64),tf.cast(test_out, tf.float32),  1)

<tf.Tensor: id=5974, shape=(10000,), dtype=bool, numpy=array([False, False, False, ..., False, False, False])>

In [4]:
x_train, y_train, x_test, y_test, _ = get_data(params)

input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
model = models.Sequential()
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=input_shape, padding = 'same'))
model.add(layers.Conv1D(64, (3), activation='relu',padding = 'same'))
model.add(layers.Conv1D(64, (3), activation='relu', padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))


In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 28, 64)            5440      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 28, 64)            12352     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 28, 64)            12352     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                17930     
Total params: 48,074
Trainable params: 48,074
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 191us/sample - loss: 0.1715 - accuracy: 0.9482 - val_loss: 0.0737 - val_accuracy: 0.9749
Epoch 2/10
60000/60000 [==============================] - 11s 178us/sample - loss: 0.0612 - accuracy: 0.9809 - val_loss: 0.0496 - val_accuracy: 0.9831
Epoch 3/10
60000/60000 [==============================] - 10s 175us/sample - loss: 0.0441 - accuracy: 0.9861 - val_loss: 0.0493 - val_accuracy: 0.9853
Epoch 4/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.0359 - accuracy: 0.9885 - val_loss: 0.0391 - val_accuracy: 0.9885
Epoch 5/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0269 - accuracy: 0.9911 - val_loss: 0.0331 - val_accuracy: 0.9904
Epoch 6/10
60000/60000 [==============================] - 10s 174us/sample - loss: 0.0243 - accuracy: 0.9921 - val_loss: 0.0443 - val_accuracy: 0.9871
Epoch 7/10
60000/60000 [====================

In [3]:
from utils import *
x_train, y_train, x_test, y_test= get_data('cifar10')
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]

In [6]:
from tensorflow.keras import layers, models

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation= None, input_shape=input_shape, padding = 'same'))
model.add(layers.Conv2D(64, (3,3), activation='relu',padding = 'same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(3, (3,3), activation='relu', padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 3)         1731      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                49168     
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 276s 6ms/sample - loss: 1.7350 - accuracy: 0.3648 - val_loss: 1.3952 - val_accuracy: 0.4992
Epoch 2/10
50000/50000 [==============================] - 282s 6ms/sample - loss: 1.2712 - accuracy: 0.5443 - val_loss: 1.2697 - val_accuracy: 0.5537
Epoch 3/10
50000/50000 [==============================] - 297s 6ms/sample - loss: 1.1055 - accuracy: 0.6046 - val_loss: 1.1255 - val_accuracy: 0.5987
Epoch 4/10
44032/50000 [=========================>....] - ETA: 33s - loss: 1.0069 - accuracy: 0.6399

KeyboardInterrupt: 